In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy
from PIL import Image
import io

In [3]:
K = 64
batch_size = 64
device = 'cuda'

In [4]:
class OM_Encoder(nn.Module):
    def __init__(self):
        super(OM_Encoder,self).__init__()
        self.fc1 = nn.Linear(3,128)
        self.fc2 = nn.Linear(128,128)
        self.fc3 = nn.Linear(256,128)
        self.fc4 = nn.Linear(256,128)
        self.mean_fc = nn.Linear(128,128)
        self.logstddev_fc = nn.Linear(128,128)
    
    def forward(self,x):
        x = x.squeeze()
        n, c, k = x.size()
        x = x.permute(0,2,1)

        x = F.relu(self.fc1(x))

        x = self.fc2(x)
        
        n,k,c = x.size()
        x = x.permute(0,2,1)

        pooled = F.max_pool1d(x, k).expand(x.size())
        x = torch.cat([x,pooled],dim=1)

        x = x.permute(0,2,1)

        x = F.relu(x)

        x = self.fc3(x)

        n,k,c = x.size()

        x = x.permute(0,2,1)

        pooled = F.max_pool1d(x, k)
        pooled = pooled.expand(x.size())

        x = torch.cat([x,pooled],dim=1)

        x = x.permute(0,2,1)

        x = F.relu(x)
        x = self.fc4(x)

        n,k,c = x.size()

        x = x.permute(0,2,1)

        x = F.max_pool1d(x, k)
        
        x= x.squeeze()

        mean = self.mean_fc(x)
        stddev = self.logstddev_fc(x)
        
        return mean,stddev

In [38]:
enc = OM_Encoder()
x,y = enc(torch.rand(64,3,100,1))
print(x.size())
print(y.size())


torch.Size([64, 3, 100, 1])
torch.Size([64, 3, 100])
torch.Size([64, 100, 3])
torch.Size([64, 100, 128])
torch.Size([64, 100, 128])
torch.Size([64, 128, 100])
torch.Size([64, 128, 100])
torch.Size([64, 256, 100])
torch.Size([64, 100, 256])
torch.Size([64, 100, 128])
torch.Size([64, 128, 100])
pooled: torch.Size([64, 128, 1])
pooled: torch.Size([64, 128, 100])
torch.Size([64, 256, 100])
torch.Size([64, 100, 256])
torch.Size([64, 100, 128])
torch.Size([64, 128, 100])
torch.Size([64, 128, 1])
torch.Size([64, 128])
torch.Size([64, 128])
torch.Size([64, 128])


In [5]:
class Block(nn.Module):
    def __init__(self):
        super(Block,self).__init__()
        self.fc1 = nn.Conv2d(256,256,kernel_size=1)
        self.fc2 = nn.Conv2d(256,256,kernel_size=1)
        self.bn1 = nn.BatchNorm2d(256, affine=False, track_running_stats=True)
        self.bn2 = nn.BatchNorm2d(256, affine=False, track_running_stats=True)
        self.gammaLayer1 = nn.Conv1d(128,256,kernel_size=1)
        self.gammaLayer2 = nn.Conv1d(128,256,kernel_size=1)
        self.betaLayer1 = nn.Conv1d(128,256,kernel_size=1)
        self.betaLayer2 = nn.Conv1d(128,256,kernel_size=1)
        
    def forward(self,y):
        x = y['ex']
        n,c,k,d = x.size()

        encoding = y['enc']
        gamma = self.gammaLayer1(encoding)

        #Need to stack the beta and gamma
        #so that we multiply all the points for one mesh
        #by the same value
        gamma = torch.stack([gamma for _ in range(k)],dim=2)
        
        beta = self.betaLayer1(encoding)
        beta = torch.stack([beta for _ in range(k)],dim=2)

        #First apply Conditional Batch Normalization
        out = gamma*self.bn1(x) + beta
        #Then ReLU activation function
        out = F.relu(out)
        #fully connected layer
        out = self.fc1(out)
        #Second CBN layer
        gamma = self.gammaLayer2(encoding)
        gamma = torch.stack([gamma for _ in range(k)],dim=2)

        beta = self.betaLayer2(encoding)
        beta = torch.stack([beta for _ in range(k)],dim=2)
        
        out = gamma* self.bn2(out) + beta
        #RELU activation
        out = F.relu(out)
        #2nd fully connected
        out = self.fc2(out)
        #Add to the input of the ResNet Block 
        out = x + out
        
        return {'ex':out, 'enc':encoding}

In [17]:
class OccupancyModel(nn.Module):
    def __init__(self):
        super(OccupancyModel,self).__init__()
        self.blocks = self.makeBlocks()
        self.encoderModel = OM_Encoder()
        self.gammaLayer = nn.Conv1d(128,256,kernel_size=1)
        self.betaLayer = nn.Conv1d(128,256,kernel_size=1)
        self.cbn = nn.BatchNorm2d(256, affine=False, track_running_stats=True)
        self.fc1 = nn.Conv2d(3,256,kernel_size=1)
        self.fc2 = nn.Conv2d(256,1,kernel_size=1)
        
    def makeBlocks(self):
        blocks = []
        for _ in range(5):
            blocks.append(Block())
        return nn.Sequential(*blocks)
   
    def sampleFromZDist(self, z):
        mean, logstddev = z
        std = logstddev.mul(0.5).exp_()
        eps = torch.randn_like(logstddev,requires_grad=True)
        return eps.mul(std).add_(mean)
        
    def forward(self,x, z_eval=None):
        if self.training:
            z_dist = self.encoderModel(x)
            z = self.sampleFromZDist(z_dist)
            z = z.unsqueeze(-1)
        else:
            z = z_eval
            z_dist = (0,1)
        x = self.fc1(x)
        #5 pre-activation ResNet-blocks
        x = self.blocks({'enc':z, 'ex':x })
        x = x['ex']
        n,c,k,d = x.size()
        
        #CBN
        gamma = self.gammaLayer(z)
        
        gamma = torch.stack([gamma for _ in range(k)],dim=2)
        
        beta = self.betaLayer(z)
        beta = torch.stack([beta for _ in range(k)],dim=2)

        x = gamma.mul(self.cbn(x)).add_(beta)
        x = F.relu(x)
        x = self.fc2(x)
        #x = x.view(-1,1)
        #x = torch.sigmoid(x)
        return x, z_dist

In [43]:
#om = OccupancyModel()
#optimizer = optim.Adam(om.parameters(), lr = 0.001)
#om.train()
optimizer.zero_grad()
pred, z_dist = om(torch.rand(5,3,100,1))
occloss = nn.BCELoss()
l1 = occloss(pred, torch.zeros_like(pred))
print(f"occloss: {l1}")
#print(z_dist[1])
#l2 = -0.5*torch.sum(1+z_dist[1] + z_dist[0].pow(2) - z_dist[1].exp())/(5*100)
mu, log_var = z_dist
l2 = mu.pow(2).add_(log_var.exp()).mul_(-1).add_(1).add_(log_var)
l2 = torch.sum(l2).mul_(-0.5)
#print(z_dist[0])
#encoderLoss = nn.KLDivLoss(reduction='batchmean')
#targetNormal = torch.stack((torch.zeros_like(z_dist[0]),torch.ones_like(z_dist[1])))
#encloss = encoderLoss(torch.stack(z_dist),targetNormal)
#print(f"encloss: {encloss}")
#print(z_dist[1])
print(l2)
loss = (l2 +  l1)/5
loss.backward()
optimizer.step()
#print(z_dist)
print(loss)

occloss: 0.16562753915786743
tensor(0.0321, grad_fn=<MulBackward0>)
tensor(0.0395, grad_fn=<DivBackward0>)


In [60]:
targetNormal = torch.stack((torch.zeros_like(z_dist[0]),torch.ones_like(z_dist[1])))


In [11]:
om = OccupancyModel()
optimizer = optim.Adam(om.parameters(), lr = 0.001)
om.train()
om.cuda()

OccupancyModel(
  (blocks): Sequential(
    (0): Block(
      (fc1): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
      (fc2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      (gammaLayer1): Conv1d(128, 256, kernel_size=(1,), stride=(1,))
      (gammaLayer2): Conv1d(128, 256, kernel_size=(1,), stride=(1,))
      (betaLayer1): Conv1d(128, 256, kernel_size=(1,), stride=(1,))
      (betaLayer2): Conv1d(128, 256, kernel_size=(1,), stride=(1,))
    )
    (1): Block(
      (fc1): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
      (fc2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      (g

In [8]:
#choose a category and load all of the available data:
import random
topdir = "/home/andrea/Documents/GradSchool/OccupancyNetworks/occupancy_networks"

#One DataSetClass per subdirectory in a category, will return "K" point samples and a single image randomly
#drawn from the 23 available
class DataSetClass(torch.utils.data.Dataset):
    def __init__(self, d):
        self.dir = d
        with numpy.load(f"{d}/points.npz") as data:
            self.pts = torch.tensor(data["points"], dtype=torch.float)
            self.occupancies = torch.tensor(numpy.unpackbits(data["occupancies"])[:self.pts.size()[0]], dtype=torch.float)
        self.K = K 
        self.length = int(self.occupancies.size()[0]/self.K)
    def __len__(self):
        return self.length
    
    def __getitem__(self,idx):
        return self.pts[idx*self.K:(idx*self.K+self.K)], self.occupancies[idx*self.K:(idx*self.K+self.K)]

       
#catalogue all of the directories with the chosen category
trainingDirs = []
couchesDirectory=f"{topdir}/data/ShapeNet/04256520"
#couchesDirectory=f"{topdir}/data/ShapeNet/02828884"
with io.open(f"{couchesDirectory}/train.lst") as trainlist:
    for traindir in trainlist.readlines():
        trainingDirs.append(f"{couchesDirectory}/{traindir.strip()}")
dataSets = []
for tdir in trainingDirs:
    dataSets.append(DataSetClass(tdir))
data = torch.utils.data.ConcatDataset(dataSets)
train_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True)
#Get the validation data
valDirs = []
with io.open(f"{couchesDirectory}/val.lst") as vallist:
    for valdir in vallist.readlines():
        valDirs.append(f"{couchesDirectory}/{valdir.strip()}")
dataSets = []
for vdir in valDirs:
    dataSets.append(DataSetClass(vdir))
val_data = torch.utils.data.ConcatDataset(dataSets)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=True)

In [9]:
def train(epoch, model, trainloader, optimizer):
    decoderLoss = nn.BCEWithLogitsLoss(reduction='sum')
    #encoderLoss = nn.KLDivLoss(reduction='batchmean')

    model.train()
    for batch_idx, data in enumerate(train_loader):
        (pts, occupancies) = data
        #Each batch size contains batch_size sets of "K" points
        pts = pts.view(-1,K, 3,1).permute(0,2,1,3).cuda()
        occupancies = occupancies.view(-1,K,1).cuda()
        optimizer.zero_grad()
        
        pred,z_dist = model(pts) #a probability for each point, and the dist parameters of latent distribution
        pred = pred.permute(0,2,1,3).squeeze(-1)
        #targetNormal = torch.stack((torch.zeros_like(z_dist[0]),torch.ones_like(z_dist[1])))
        #encloss = encoderLoss(torch.stack(z_dist),targetNormal)
        mu, log_var = z_dist
        encloss = mu.pow(2).add_(log_var.exp()).mul_(-1).add_(1).add_(log_var)
        encloss = torch.sum(encloss).mul_(-0.5)
        #encloss = -0.5*torch.sum(1+z_dist[1] + z_dist[0].pow(2) - z_dist[1].exp())
        #print(encloss)
        decloss = decoderLoss(pred,occupancies)
        
        loss = (encloss + decloss/K)/batch_size
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader), 100. * batch_idx / len(train_loader),
                loss.item()))
            #print(f"Reconstruction Loss {decloss/(K*batch_size)}")
        if batch_idx % 100 == 0:
            print("Saving to model3.pth")
            torch.save(model.state_dict(), "unconditional_model3.pth")

In [118]:
train(0,om,train_loader,optimizer)

Train Epoch: 0 [0/54231 (0%)]\tLoss: 1.054565
Saving to model3.pth
Train Epoch: 0 [20/54231 (0%)]\tLoss: 0.319196
Train Epoch: 0 [40/54231 (0%)]\tLoss: 0.258088
Train Epoch: 0 [60/54231 (0%)]\tLoss: 0.267797
Train Epoch: 0 [80/54231 (0%)]\tLoss: 0.303531
Train Epoch: 0 [100/54231 (0%)]\tLoss: 0.242265
Train Epoch: 0 [120/54231 (0%)]\tLoss: 0.240289
Train Epoch: 0 [140/54231 (0%)]\tLoss: 0.256170
Train Epoch: 0 [160/54231 (0%)]\tLoss: 0.289165
Train Epoch: 0 [180/54231 (0%)]\tLoss: 0.254502
Train Epoch: 0 [200/54231 (0%)]\tLoss: 0.274218
Saving to model3.pth
Train Epoch: 0 [220/54231 (0%)]\tLoss: 0.260413
Train Epoch: 0 [240/54231 (0%)]\tLoss: 0.236656
Train Epoch: 0 [260/54231 (0%)]\tLoss: 0.264093
Train Epoch: 0 [280/54231 (0%)]\tLoss: 0.273885
Train Epoch: 0 [300/54231 (0%)]\tLoss: 0.238217
Train Epoch: 0 [320/54231 (0%)]\tLoss: 0.230864
Train Epoch: 0 [340/54231 (0%)]\tLoss: 0.249316
Train Epoch: 0 [360/54231 (0%)]\tLoss: 0.266711
Train Epoch: 0 [380/54231 (0%)]\tLoss: 0.246309
Trai

Train Epoch: 0 [3240/54231 (3%)]\tLoss: 0.140688
Train Epoch: 0 [3260/54231 (3%)]\tLoss: 0.166323
Train Epoch: 0 [3280/54231 (3%)]\tLoss: 0.162355
Train Epoch: 0 [3300/54231 (3%)]\tLoss: 0.195731
Train Epoch: 0 [3320/54231 (3%)]\tLoss: 0.175904
Train Epoch: 0 [3340/54231 (3%)]\tLoss: 0.202630
Train Epoch: 0 [3360/54231 (3%)]\tLoss: 0.164397
Train Epoch: 0 [3380/54231 (3%)]\tLoss: 0.184988
Train Epoch: 0 [3400/54231 (3%)]\tLoss: 0.138204
Saving to model3.pth
Train Epoch: 0 [3420/54231 (3%)]\tLoss: 0.137767
Train Epoch: 0 [3440/54231 (3%)]\tLoss: 0.138214
Train Epoch: 0 [3460/54231 (3%)]\tLoss: 0.151230
Train Epoch: 0 [3480/54231 (3%)]\tLoss: 0.164579
Train Epoch: 0 [3500/54231 (3%)]\tLoss: 0.146862
Train Epoch: 0 [3520/54231 (3%)]\tLoss: 0.166501
Train Epoch: 0 [3540/54231 (3%)]\tLoss: 0.185496
Train Epoch: 0 [3560/54231 (3%)]\tLoss: 0.161220
Train Epoch: 0 [3580/54231 (3%)]\tLoss: 0.149139
Train Epoch: 0 [3600/54231 (3%)]\tLoss: 0.144617
Saving to model3.pth
Train Epoch: 0 [3620/54231 

Train Epoch: 0 [6460/54231 (6%)]\tLoss: 0.144745
Train Epoch: 0 [6480/54231 (6%)]\tLoss: 0.162788
Train Epoch: 0 [6500/54231 (6%)]\tLoss: 0.165743
Train Epoch: 0 [6520/54231 (6%)]\tLoss: 0.171918
Train Epoch: 0 [6540/54231 (6%)]\tLoss: 0.166659
Train Epoch: 0 [6560/54231 (6%)]\tLoss: 0.130787
Train Epoch: 0 [6580/54231 (6%)]\tLoss: 0.173580
Train Epoch: 0 [6600/54231 (6%)]\tLoss: 0.167207
Saving to model3.pth
Train Epoch: 0 [6620/54231 (6%)]\tLoss: 0.166291
Train Epoch: 0 [6640/54231 (6%)]\tLoss: 0.198502
Train Epoch: 0 [6660/54231 (6%)]\tLoss: 0.146684
Train Epoch: 0 [6680/54231 (6%)]\tLoss: 0.141930
Train Epoch: 0 [6700/54231 (6%)]\tLoss: 0.136395
Train Epoch: 0 [6720/54231 (6%)]\tLoss: 0.139176
Train Epoch: 0 [6740/54231 (6%)]\tLoss: 0.149128
Train Epoch: 0 [6760/54231 (6%)]\tLoss: 0.135778
Train Epoch: 0 [6780/54231 (6%)]\tLoss: 0.182012
Train Epoch: 0 [6800/54231 (6%)]\tLoss: 0.165522
Saving to model3.pth
Train Epoch: 0 [6820/54231 (6%)]\tLoss: 0.161506
Train Epoch: 0 [6840/54231 

Train Epoch: 0 [9680/54231 (9%)]\tLoss: 0.148569
Train Epoch: 0 [9700/54231 (9%)]\tLoss: 0.169565
Train Epoch: 0 [9720/54231 (9%)]\tLoss: 0.177874
Train Epoch: 0 [9740/54231 (9%)]\tLoss: 0.182745
Train Epoch: 0 [9760/54231 (9%)]\tLoss: 0.160002
Train Epoch: 0 [9780/54231 (9%)]\tLoss: 0.139013
Train Epoch: 0 [9800/54231 (9%)]\tLoss: 0.133226
Saving to model3.pth
Train Epoch: 0 [9820/54231 (9%)]\tLoss: 0.179584
Train Epoch: 0 [9840/54231 (9%)]\tLoss: 0.157485
Train Epoch: 0 [9860/54231 (9%)]\tLoss: 0.153984
Train Epoch: 0 [9880/54231 (9%)]\tLoss: 0.150422
Train Epoch: 0 [9900/54231 (9%)]\tLoss: 0.169636
Train Epoch: 0 [9920/54231 (9%)]\tLoss: 0.147457
Train Epoch: 0 [9940/54231 (9%)]\tLoss: 0.139240
Train Epoch: 0 [9960/54231 (9%)]\tLoss: 0.179895
Train Epoch: 0 [9980/54231 (9%)]\tLoss: 0.128808
Train Epoch: 0 [10000/54231 (9%)]\tLoss: 0.168766
Saving to model3.pth
Train Epoch: 0 [10020/54231 (9%)]\tLoss: 0.134391
Train Epoch: 0 [10040/54231 (9%)]\tLoss: 0.163238
Train Epoch: 0 [10060/54

Train Epoch: 0 [12800/54231 (12%)]\tLoss: 0.168959
Saving to model3.pth
Train Epoch: 0 [12820/54231 (12%)]\tLoss: 0.153981
Train Epoch: 0 [12840/54231 (12%)]\tLoss: 0.138812
Train Epoch: 0 [12860/54231 (12%)]\tLoss: 0.174088
Train Epoch: 0 [12880/54231 (12%)]\tLoss: 0.150221
Train Epoch: 0 [12900/54231 (12%)]\tLoss: 0.155215
Train Epoch: 0 [12920/54231 (12%)]\tLoss: 0.198528
Train Epoch: 0 [12940/54231 (12%)]\tLoss: 0.157952
Train Epoch: 0 [12960/54231 (12%)]\tLoss: 0.156537
Train Epoch: 0 [12980/54231 (12%)]\tLoss: 0.156846
Train Epoch: 0 [13000/54231 (12%)]\tLoss: 0.169422
Saving to model3.pth
Train Epoch: 0 [13020/54231 (12%)]\tLoss: 0.161548
Train Epoch: 0 [13040/54231 (12%)]\tLoss: 0.146566
Train Epoch: 0 [13060/54231 (12%)]\tLoss: 0.173913
Train Epoch: 0 [13080/54231 (12%)]\tLoss: 0.143445
Train Epoch: 0 [13100/54231 (12%)]\tLoss: 0.151441
Train Epoch: 0 [13120/54231 (12%)]\tLoss: 0.128146
Train Epoch: 0 [13140/54231 (12%)]\tLoss: 0.173532
Train Epoch: 0 [13160/54231 (12%)]\tLoss

...

Train Epoch: 0 [93400/54231 (86%)]\tLoss: 0.148455
Saving to model3.pth
Train Epoch: 0 [93420/54231 (86%)]\tLoss: 0.155832
Train Epoch: 0 [93440/54231 (86%)]\tLoss: 0.122953
Train Epoch: 0 [93460/54231 (86%)]\tLoss: 0.149423
Train Epoch: 0 [93480/54231 (86%)]\tLoss: 0.136133
Train Epoch: 0 [93500/54231 (86%)]\tLoss: 0.139215
Train Epoch: 0 [93520/54231 (86%)]\tLoss: 0.148618
Train Epoch: 0 [93540/54231 (86%)]\tLoss: 0.145272
Train Epoch: 0 [93560/54231 (86%)]\tLoss: 0.199272
Train Epoch: 0 [93580/54231 (86%)]\tLoss: 0.115881
Train Epoch: 0 [93600/54231 (86%)]\tLoss: 0.159581
Saving to model3.pth
Train Epoch: 0 [93620/54231 (86%)]\tLoss: 0.199877
Train Epoch: 0 [93640/54231 (86%)]\tLoss: 0.199750
Train Epoch: 0 [93660/54231 (86%)]\tLoss: 0.120955
Train Epoch: 0 [93680/54231 (86%)]\tLoss: 0.136571
Train Epoch: 0 [93700/54231 (86%)]\tLoss: 0.146598
Train Epoch: 0 [93720/54231 (86%)]\tLoss: 0.131623
Train Epoch: 0 [93740/54231 (86%)]\tLoss: 0.128197
Train Epoch: 0 [93760/54231 (86%)]\tLoss

Train Epoch: 0 [96500/54231 (89%)]\tLoss: 0.142570
Train Epoch: 0 [96520/54231 (89%)]\tLoss: 0.133352
Train Epoch: 0 [96540/54231 (89%)]\tLoss: 0.152342
Train Epoch: 0 [96560/54231 (89%)]\tLoss: 0.139540
Train Epoch: 0 [96580/54231 (89%)]\tLoss: 0.123321
Train Epoch: 0 [96600/54231 (89%)]\tLoss: 0.126296
Saving to model3.pth
Train Epoch: 0 [96620/54231 (89%)]\tLoss: 0.147217
Train Epoch: 0 [96640/54231 (89%)]\tLoss: 0.125723
Train Epoch: 0 [96660/54231 (89%)]\tLoss: 0.128675
Train Epoch: 0 [96680/54231 (89%)]\tLoss: 0.134597
Train Epoch: 0 [96700/54231 (89%)]\tLoss: 0.129037
Train Epoch: 0 [96720/54231 (89%)]\tLoss: 0.152687
Train Epoch: 0 [96740/54231 (89%)]\tLoss: 0.139082
Train Epoch: 0 [96760/54231 (89%)]\tLoss: 0.152332
Train Epoch: 0 [96780/54231 (89%)]\tLoss: 0.184392
Train Epoch: 0 [96800/54231 (89%)]\tLoss: 0.176038
Saving to model3.pth
Train Epoch: 0 [96820/54231 (89%)]\tLoss: 0.142963
Train Epoch: 0 [96840/54231 (89%)]\tLoss: 0.154002
Train Epoch: 0 [96860/54231 (89%)]\tLoss

Train Epoch: 0 [99600/54231 (92%)]\tLoss: 0.160424
Saving to model3.pth
Train Epoch: 0 [99620/54231 (92%)]\tLoss: 0.153175
Train Epoch: 0 [99640/54231 (92%)]\tLoss: 0.124527
Train Epoch: 0 [99660/54231 (92%)]\tLoss: 0.161759
Train Epoch: 0 [99680/54231 (92%)]\tLoss: 0.159660
Train Epoch: 0 [99700/54231 (92%)]\tLoss: 0.140543
Train Epoch: 0 [99720/54231 (92%)]\tLoss: 0.122947
Train Epoch: 0 [99740/54231 (92%)]\tLoss: 0.133596
Train Epoch: 0 [99760/54231 (92%)]\tLoss: 0.135770
Train Epoch: 0 [99780/54231 (92%)]\tLoss: 0.137669
Train Epoch: 0 [99800/54231 (92%)]\tLoss: 0.138465
Saving to model3.pth
Train Epoch: 0 [99820/54231 (92%)]\tLoss: 0.174782
Train Epoch: 0 [99840/54231 (92%)]\tLoss: 0.121503
Train Epoch: 0 [99860/54231 (92%)]\tLoss: 0.139439
Train Epoch: 0 [99880/54231 (92%)]\tLoss: 0.124041
Train Epoch: 0 [99900/54231 (92%)]\tLoss: 0.151448
Train Epoch: 0 [99920/54231 (92%)]\tLoss: 0.157767
Train Epoch: 0 [99940/54231 (92%)]\tLoss: 0.127622
Train Epoch: 0 [99960/54231 (92%)]\tLoss

Train Epoch: 0 [102640/54231 (95%)]\tLoss: 0.188514
Train Epoch: 0 [102660/54231 (95%)]\tLoss: 0.155866
Train Epoch: 0 [102680/54231 (95%)]\tLoss: 0.122877
Train Epoch: 0 [102700/54231 (95%)]\tLoss: 0.122694
Train Epoch: 0 [102720/54231 (95%)]\tLoss: 0.135497
Train Epoch: 0 [102740/54231 (95%)]\tLoss: 0.155399
Train Epoch: 0 [102760/54231 (95%)]\tLoss: 0.146584
Train Epoch: 0 [102780/54231 (95%)]\tLoss: 0.139469
Train Epoch: 0 [102800/54231 (95%)]\tLoss: 0.138376
Saving to model3.pth
Train Epoch: 0 [102820/54231 (95%)]\tLoss: 0.163318
Train Epoch: 0 [102840/54231 (95%)]\tLoss: 0.140497
Train Epoch: 0 [102860/54231 (95%)]\tLoss: 0.156996
Train Epoch: 0 [102880/54231 (95%)]\tLoss: 0.148327
Train Epoch: 0 [102900/54231 (95%)]\tLoss: 0.117334
Train Epoch: 0 [102920/54231 (95%)]\tLoss: 0.157817
Train Epoch: 0 [102940/54231 (95%)]\tLoss: 0.154702
Train Epoch: 0 [102960/54231 (95%)]\tLoss: 0.149660
Train Epoch: 0 [102980/54231 (95%)]\tLoss: 0.115783
Train Epoch: 0 [103000/54231 (95%)]\tLoss: 

Train Epoch: 0 [105680/54231 (97%)]\tLoss: 0.118144
Train Epoch: 0 [105700/54231 (97%)]\tLoss: 0.146106
Train Epoch: 0 [105720/54231 (97%)]\tLoss: 0.180058
Train Epoch: 0 [105740/54231 (97%)]\tLoss: 0.113485
Train Epoch: 0 [105760/54231 (98%)]\tLoss: 0.137856
Train Epoch: 0 [105780/54231 (98%)]\tLoss: 0.184883
Train Epoch: 0 [105800/54231 (98%)]\tLoss: 0.140768
Saving to model3.pth
Train Epoch: 0 [105820/54231 (98%)]\tLoss: 0.152589
Train Epoch: 0 [105840/54231 (98%)]\tLoss: 0.141393
Train Epoch: 0 [105860/54231 (98%)]\tLoss: 0.119106
Train Epoch: 0 [105880/54231 (98%)]\tLoss: 0.133917
Train Epoch: 0 [105900/54231 (98%)]\tLoss: 0.145729
Train Epoch: 0 [105920/54231 (98%)]\tLoss: 0.134807
Train Epoch: 0 [105940/54231 (98%)]\tLoss: 0.138983
Train Epoch: 0 [105960/54231 (98%)]\tLoss: 0.136120
Train Epoch: 0 [105980/54231 (98%)]\tLoss: 0.131821
Train Epoch: 0 [106000/54231 (98%)]\tLoss: 0.151141
Saving to model3.pth
Train Epoch: 0 [106020/54231 (98%)]\tLoss: 0.107110
Train Epoch: 0 [106040

In [119]:
for ep in range(1,10):
    train(ep,om,train_loader,optimizer)

Train Epoch: 1 [0/54231 (0%)]\tLoss: 0.127561
Saving to model3.pth
Train Epoch: 1 [20/54231 (0%)]\tLoss: 0.167288
Train Epoch: 1 [40/54231 (0%)]\tLoss: 0.137448
Train Epoch: 1 [60/54231 (0%)]\tLoss: 0.150859
Train Epoch: 1 [80/54231 (0%)]\tLoss: 0.115513
Train Epoch: 1 [100/54231 (0%)]\tLoss: 0.122381
Train Epoch: 1 [120/54231 (0%)]\tLoss: 0.130459
Train Epoch: 1 [140/54231 (0%)]\tLoss: 0.119512
Train Epoch: 1 [160/54231 (0%)]\tLoss: 0.146059
Train Epoch: 1 [180/54231 (0%)]\tLoss: 0.144824
Train Epoch: 1 [200/54231 (0%)]\tLoss: 0.173934
Saving to model3.pth
Train Epoch: 1 [220/54231 (0%)]\tLoss: 0.140921
Train Epoch: 1 [240/54231 (0%)]\tLoss: 0.127235
Train Epoch: 1 [260/54231 (0%)]\tLoss: 0.129344
Train Epoch: 1 [280/54231 (0%)]\tLoss: 0.187076
Train Epoch: 1 [300/54231 (0%)]\tLoss: 0.155341
Train Epoch: 1 [320/54231 (0%)]\tLoss: 0.122928
Train Epoch: 1 [340/54231 (0%)]\tLoss: 0.146774
Train Epoch: 1 [360/54231 (0%)]\tLoss: 0.121066
Train Epoch: 1 [380/54231 (0%)]\tLoss: 0.167207
Trai

Train Epoch: 1 [3240/54231 (3%)]\tLoss: 0.136379
Train Epoch: 1 [3260/54231 (3%)]\tLoss: 0.155764
Train Epoch: 1 [3280/54231 (3%)]\tLoss: 0.124739
Train Epoch: 1 [3300/54231 (3%)]\tLoss: 0.137196
Train Epoch: 1 [3320/54231 (3%)]\tLoss: 0.188150
Train Epoch: 1 [3340/54231 (3%)]\tLoss: 0.133077
Train Epoch: 1 [3360/54231 (3%)]\tLoss: 0.140869
Train Epoch: 1 [3380/54231 (3%)]\tLoss: 0.144970
Train Epoch: 1 [3400/54231 (3%)]\tLoss: 0.142745
Saving to model3.pth
Train Epoch: 1 [3420/54231 (3%)]\tLoss: 0.150849
Train Epoch: 1 [3440/54231 (3%)]\tLoss: 0.168603
Train Epoch: 1 [3460/54231 (3%)]\tLoss: 0.134010
Train Epoch: 1 [3480/54231 (3%)]\tLoss: 0.174604
Train Epoch: 1 [3500/54231 (3%)]\tLoss: 0.141066
Train Epoch: 1 [3520/54231 (3%)]\tLoss: 0.163352
Train Epoch: 1 [3540/54231 (3%)]\tLoss: 0.141094
Train Epoch: 1 [3560/54231 (3%)]\tLoss: 0.152394
Train Epoch: 1 [3580/54231 (3%)]\tLoss: 0.127444
Train Epoch: 1 [3600/54231 (3%)]\tLoss: 0.118743
Saving to model3.pth
Train Epoch: 1 [3620/54231 

Train Epoch: 1 [6460/54231 (6%)]\tLoss: 0.152667
Train Epoch: 1 [6480/54231 (6%)]\tLoss: 0.140791
Train Epoch: 1 [6500/54231 (6%)]\tLoss: 0.134238
Train Epoch: 1 [6520/54231 (6%)]\tLoss: 0.160693
Train Epoch: 1 [6540/54231 (6%)]\tLoss: 0.156176
Train Epoch: 1 [6560/54231 (6%)]\tLoss: 0.128879
Train Epoch: 1 [6580/54231 (6%)]\tLoss: 0.178138
Train Epoch: 1 [6600/54231 (6%)]\tLoss: 0.167343
Saving to model3.pth
Train Epoch: 1 [6620/54231 (6%)]\tLoss: 0.131129
Train Epoch: 1 [6640/54231 (6%)]\tLoss: 0.146968
Train Epoch: 1 [6660/54231 (6%)]\tLoss: 0.162401
Train Epoch: 1 [6680/54231 (6%)]\tLoss: 0.137999
Train Epoch: 1 [6700/54231 (6%)]\tLoss: 0.135186
Train Epoch: 1 [6720/54231 (6%)]\tLoss: 0.163766
Train Epoch: 1 [6740/54231 (6%)]\tLoss: 0.145206
Train Epoch: 1 [6760/54231 (6%)]\tLoss: 0.156081
Train Epoch: 1 [6780/54231 (6%)]\tLoss: 0.151546
Train Epoch: 1 [6800/54231 (6%)]\tLoss: 0.134429
Saving to model3.pth
Train Epoch: 1 [6820/54231 (6%)]\tLoss: 0.146536
Train Epoch: 1 [6840/54231 

Train Epoch: 1 [9680/54231 (9%)]\tLoss: 0.116669
Train Epoch: 1 [9700/54231 (9%)]\tLoss: 0.126016
Train Epoch: 1 [9720/54231 (9%)]\tLoss: 0.144396
Train Epoch: 1 [9740/54231 (9%)]\tLoss: 0.155085
Train Epoch: 1 [9760/54231 (9%)]\tLoss: 0.203201
Train Epoch: 1 [9780/54231 (9%)]\tLoss: 0.132298
Train Epoch: 1 [9800/54231 (9%)]\tLoss: 0.151487
Saving to model3.pth
Train Epoch: 1 [9820/54231 (9%)]\tLoss: 0.108882
Train Epoch: 1 [9840/54231 (9%)]\tLoss: 0.145489
Train Epoch: 1 [9860/54231 (9%)]\tLoss: 0.151941
Train Epoch: 1 [9880/54231 (9%)]\tLoss: 0.129871
Train Epoch: 1 [9900/54231 (9%)]\tLoss: 0.142463
Train Epoch: 1 [9920/54231 (9%)]\tLoss: 0.130664
Train Epoch: 1 [9940/54231 (9%)]\tLoss: 0.125497
Train Epoch: 1 [9960/54231 (9%)]\tLoss: 0.133174
Train Epoch: 1 [9980/54231 (9%)]\tLoss: 0.138667
Train Epoch: 1 [10000/54231 (9%)]\tLoss: 0.142862
Saving to model3.pth
Train Epoch: 1 [10020/54231 (9%)]\tLoss: 0.123362
Train Epoch: 1 [10040/54231 (9%)]\tLoss: 0.127593
Train Epoch: 1 [10060/54

Train Epoch: 1 [12800/54231 (12%)]\tLoss: 0.142570
Saving to model3.pth
Train Epoch: 1 [12820/54231 (12%)]\tLoss: 0.148885
Train Epoch: 1 [12840/54231 (12%)]\tLoss: 0.148888
Train Epoch: 1 [12860/54231 (12%)]\tLoss: 0.157750
Train Epoch: 1 [12880/54231 (12%)]\tLoss: 0.159279
Train Epoch: 1 [12900/54231 (12%)]\tLoss: 0.181629
Train Epoch: 1 [12920/54231 (12%)]\tLoss: 0.204230
Train Epoch: 1 [12940/54231 (12%)]\tLoss: 0.140090
Train Epoch: 1 [12960/54231 (12%)]\tLoss: 0.144500
Train Epoch: 1 [12980/54231 (12%)]\tLoss: 0.141362
Train Epoch: 1 [13000/54231 (12%)]\tLoss: 0.137288
Saving to model3.pth
Train Epoch: 1 [13020/54231 (12%)]\tLoss: 0.137631
Train Epoch: 1 [13040/54231 (12%)]\tLoss: 0.194425
Train Epoch: 1 [13060/54231 (12%)]\tLoss: 0.134197
Train Epoch: 1 [13080/54231 (12%)]\tLoss: 0.164244
Train Epoch: 1 [13100/54231 (12%)]\tLoss: 0.145420
Train Epoch: 1 [13120/54231 (12%)]\tLoss: 0.133440
Train Epoch: 1 [13140/54231 (12%)]\tLoss: 0.128147
Train Epoch: 1 [13160/54231 (12%)]\tLoss

...

Train Epoch: 9 [95160/54231 (88%)]\tLoss: 0.147706
Train Epoch: 9 [95180/54231 (88%)]\tLoss: 0.150369
Train Epoch: 9 [95200/54231 (88%)]\tLoss: 0.150328
Saving to model3.pth
Train Epoch: 9 [95220/54231 (88%)]\tLoss: 0.126221
Train Epoch: 9 [95240/54231 (88%)]\tLoss: 0.127717
Train Epoch: 9 [95260/54231 (88%)]\tLoss: 0.133528
Train Epoch: 9 [95280/54231 (88%)]\tLoss: 0.112873
Train Epoch: 9 [95300/54231 (88%)]\tLoss: 0.142874
Train Epoch: 9 [95320/54231 (88%)]\tLoss: 0.131975
Train Epoch: 9 [95340/54231 (88%)]\tLoss: 0.137174
Train Epoch: 9 [95360/54231 (88%)]\tLoss: 0.154187
Train Epoch: 9 [95380/54231 (88%)]\tLoss: 0.128107
Train Epoch: 9 [95400/54231 (88%)]\tLoss: 0.151923
Saving to model3.pth
Train Epoch: 9 [95420/54231 (88%)]\tLoss: 0.154537
Train Epoch: 9 [95440/54231 (88%)]\tLoss: 0.112891
Train Epoch: 9 [95460/54231 (88%)]\tLoss: 0.110426
Train Epoch: 9 [95480/54231 (88%)]\tLoss: 0.151506
Train Epoch: 9 [95500/54231 (88%)]\tLoss: 0.134217
Train Epoch: 9 [95520/54231 (88%)]\tLoss

Train Epoch: 9 [98260/54231 (91%)]\tLoss: 0.157351
Train Epoch: 9 [98280/54231 (91%)]\tLoss: 0.131689
Train Epoch: 9 [98300/54231 (91%)]\tLoss: 0.134254
Train Epoch: 9 [98320/54231 (91%)]\tLoss: 0.129144
Train Epoch: 9 [98340/54231 (91%)]\tLoss: 0.144815
Train Epoch: 9 [98360/54231 (91%)]\tLoss: 0.121511
Train Epoch: 9 [98380/54231 (91%)]\tLoss: 0.168659
Train Epoch: 9 [98400/54231 (91%)]\tLoss: 0.113581
Saving to model3.pth
Train Epoch: 9 [98420/54231 (91%)]\tLoss: 0.127069
Train Epoch: 9 [98440/54231 (91%)]\tLoss: 0.120671
Train Epoch: 9 [98460/54231 (91%)]\tLoss: 0.142826
Train Epoch: 9 [98480/54231 (91%)]\tLoss: 0.148481
Train Epoch: 9 [98500/54231 (91%)]\tLoss: 0.149196
Train Epoch: 9 [98520/54231 (91%)]\tLoss: 0.167734
Train Epoch: 9 [98540/54231 (91%)]\tLoss: 0.130106
Train Epoch: 9 [98560/54231 (91%)]\tLoss: 0.126504
Train Epoch: 9 [98580/54231 (91%)]\tLoss: 0.130551
Train Epoch: 9 [98600/54231 (91%)]\tLoss: 0.123503
Saving to model3.pth
Train Epoch: 9 [98620/54231 (91%)]\tLoss

Train Epoch: 9 [101340/54231 (93%)]\tLoss: 0.137039
Train Epoch: 9 [101360/54231 (93%)]\tLoss: 0.126921
Train Epoch: 9 [101380/54231 (93%)]\tLoss: 0.150944
Train Epoch: 9 [101400/54231 (93%)]\tLoss: 0.134620
Saving to model3.pth
Train Epoch: 9 [101420/54231 (94%)]\tLoss: 0.136927
Train Epoch: 9 [101440/54231 (94%)]\tLoss: 0.128809
Train Epoch: 9 [101460/54231 (94%)]\tLoss: 0.133784
Train Epoch: 9 [101480/54231 (94%)]\tLoss: 0.177655
Train Epoch: 9 [101500/54231 (94%)]\tLoss: 0.125597
Train Epoch: 9 [101520/54231 (94%)]\tLoss: 0.146554
Train Epoch: 9 [101540/54231 (94%)]\tLoss: 0.141598
Train Epoch: 9 [101560/54231 (94%)]\tLoss: 0.159522
Train Epoch: 9 [101580/54231 (94%)]\tLoss: 0.152871
Train Epoch: 9 [101600/54231 (94%)]\tLoss: 0.123949
Saving to model3.pth
Train Epoch: 9 [101620/54231 (94%)]\tLoss: 0.176387
Train Epoch: 9 [101640/54231 (94%)]\tLoss: 0.136392
Train Epoch: 9 [101660/54231 (94%)]\tLoss: 0.145971
Train Epoch: 9 [101680/54231 (94%)]\tLoss: 0.138081
Train Epoch: 9 [101700

Train Epoch: 9 [104380/54231 (96%)]\tLoss: 0.171215
Train Epoch: 9 [104400/54231 (96%)]\tLoss: 0.181542
Saving to model3.pth
Train Epoch: 9 [104420/54231 (96%)]\tLoss: 0.152671
Train Epoch: 9 [104440/54231 (96%)]\tLoss: 0.183284
Train Epoch: 9 [104460/54231 (96%)]\tLoss: 0.128622
Train Epoch: 9 [104480/54231 (96%)]\tLoss: 0.159832
Train Epoch: 9 [104500/54231 (96%)]\tLoss: 0.154637
Train Epoch: 9 [104520/54231 (96%)]\tLoss: 0.130249
Train Epoch: 9 [104540/54231 (96%)]\tLoss: 0.128612
Train Epoch: 9 [104560/54231 (96%)]\tLoss: 0.114441
Train Epoch: 9 [104580/54231 (96%)]\tLoss: 0.171492
Train Epoch: 9 [104600/54231 (96%)]\tLoss: 0.117948
Saving to model3.pth
Train Epoch: 9 [104620/54231 (96%)]\tLoss: 0.138352
Train Epoch: 9 [104640/54231 (96%)]\tLoss: 0.134985
Train Epoch: 9 [104660/54231 (96%)]\tLoss: 0.142771
Train Epoch: 9 [104680/54231 (97%)]\tLoss: 0.111234
Train Epoch: 9 [104700/54231 (97%)]\tLoss: 0.147360
Train Epoch: 9 [104720/54231 (97%)]\tLoss: 0.154653
Train Epoch: 9 [104740

Train Epoch: 9 [107420/54231 (99%)]\tLoss: 0.112757
Train Epoch: 9 [107440/54231 (99%)]\tLoss: 0.122715
Train Epoch: 9 [107460/54231 (99%)]\tLoss: 0.127044
Train Epoch: 9 [107480/54231 (99%)]\tLoss: 0.119418
Train Epoch: 9 [107500/54231 (99%)]\tLoss: 0.187169
Train Epoch: 9 [107520/54231 (99%)]\tLoss: 0.168629
Train Epoch: 9 [107540/54231 (99%)]\tLoss: 0.122069
Train Epoch: 9 [107560/54231 (99%)]\tLoss: 0.152455
Train Epoch: 9 [107580/54231 (99%)]\tLoss: 0.148089
Train Epoch: 9 [107600/54231 (99%)]\tLoss: 0.163631
Saving to model3.pth
Train Epoch: 9 [107620/54231 (99%)]\tLoss: 0.144488
Train Epoch: 9 [107640/54231 (99%)]\tLoss: 0.141494
Train Epoch: 9 [107660/54231 (99%)]\tLoss: 0.163245
Train Epoch: 9 [107680/54231 (99%)]\tLoss: 0.143435
Train Epoch: 9 [107700/54231 (99%)]\tLoss: 0.121024
Train Epoch: 9 [107720/54231 (99%)]\tLoss: 0.147073
Train Epoch: 9 [107740/54231 (99%)]\tLoss: 0.132173
Train Epoch: 9 [107760/54231 (99%)]\tLoss: 0.137397
Train Epoch: 9 [107780/54231 (99%)]\tLoss: 

In [18]:
model = OccupancyModel()
model.load_state_dict(torch.load("unconditional_model3.pth",map_location=device))
model.cuda()
model.eval()

OccupancyModel(
  (blocks): Sequential(
    (0): Block(
      (fc1): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
      (fc2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      (gammaLayer1): Conv1d(128, 256, kernel_size=(1,), stride=(1,))
      (gammaLayer2): Conv1d(128, 256, kernel_size=(1,), stride=(1,))
      (betaLayer1): Conv1d(128, 256, kernel_size=(1,), stride=(1,))
      (betaLayer2): Conv1d(128, 256, kernel_size=(1,), stride=(1,))
    )
    (1): Block(
      (fc1): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
      (fc2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      (g

In [45]:
from torch.autograd import Variable
def validation(model, val_loader):
    model.eval()
    decoderLoss = nn.BCEWithLogitsLoss(reduction='mean')

    validation_loss = 0
    correct = 0
    for batch_idx, data in enumerate(val_loader):
        (pts, occupancies) = data
        pts = pts.view(-1,K, 3,1).permute(0,2,1,3).cuda()
        occupancies = occupancies.view(-1,K,1).cuda()
        z = Variable(torch.randn(pts.size()[0],128))
        z= z.unsqueeze(-1).cuda()
        pred,z_dist = model(pts,z) 
        pred = pred.permute(0,2,1,3).squeeze(-1)
        pred = torch.sigmoid(pred)
        #print(pred)
        #print(occupancies)
        loss = decoderLoss(pred, occupancies)
        validation_loss += loss.item()
        
        threshold = 0.6
        roundedOut = [1 if out > threshold else 0 for out in pred.view(-1)]
        roundedOut = torch.tensor(roundedOut).cuda()
        correctNow = roundedOut.eq(occupancies.view(-1)).sum()
        correct += correctNow
        validation_loss /= len(val_loader.dataset)
        print('Validation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        validation_loss, correctNow, pts.size()[0]*K, 100. * correctNow / (pts.size()[0]*K)))

In [46]:
validation(model, val_loader)

Validation set: Average loss: 0.0000, Accuracy: 3871/4096 (95%)

Validation set: Average loss: 0.0000, Accuracy: 3867/4096 (94%)

Validation set: Average loss: 0.0000, Accuracy: 3844/4096 (94%)

Validation set: Average loss: 0.0000, Accuracy: 3866/4096 (94%)

Validation set: Average loss: 0.0000, Accuracy: 3862/4096 (94%)

Validation set: Average loss: 0.0000, Accuracy: 3874/4096 (95%)

Validation set: Average loss: 0.0000, Accuracy: 3845/4096 (94%)

Validation set: Average loss: 0.0000, Accuracy: 3846/4096 (94%)

Validation set: Average loss: 0.0000, Accuracy: 3879/4096 (95%)

Validation set: Average loss: 0.0000, Accuracy: 3894/4096 (95%)

Validation set: Average loss: 0.0000, Accuracy: 3882/4096 (95%)

Validation set: Average loss: 0.0000, Accuracy: 3849/4096 (94%)

Validation set: Average loss: 0.0000, Accuracy: 3861/4096 (94%)

Validation set: Average loss: 0.0000, Accuracy: 3840/4096 (94%)

Validation set: Average loss: 0.0000, Accuracy: 3860/4096 (94%)

Validation set: Average l

Validation set: Average loss: 0.0000, Accuracy: 3827/4096 (93%)

Validation set: Average loss: 0.0000, Accuracy: 3878/4096 (95%)

Validation set: Average loss: 0.0000, Accuracy: 3860/4096 (94%)

Validation set: Average loss: 0.0000, Accuracy: 3867/4096 (94%)

Validation set: Average loss: 0.0000, Accuracy: 3891/4096 (95%)

Validation set: Average loss: 0.0000, Accuracy: 3835/4096 (94%)

Validation set: Average loss: 0.0000, Accuracy: 3877/4096 (95%)

Validation set: Average loss: 0.0000, Accuracy: 3858/4096 (94%)

Validation set: Average loss: 0.0000, Accuracy: 3862/4096 (94%)

Validation set: Average loss: 0.0000, Accuracy: 3871/4096 (95%)

Validation set: Average loss: 0.0000, Accuracy: 3873/4096 (95%)

Validation set: Average loss: 0.0000, Accuracy: 3863/4096 (94%)

Validation set: Average loss: 0.0000, Accuracy: 3881/4096 (95%)

Validation set: Average loss: 0.0000, Accuracy: 3873/4096 (95%)

Validation set: Average loss: 0.0000, Accuracy: 3873/4096 (95%)

Validation set: Average l

KeyboardInterrupt: 

In [2]:
import torch
torch.rand((1,128))

tensor([[0.8843, 0.5049, 0.5723, 0.8481, 0.5566, 0.5038, 0.3710, 0.8644, 0.3512,
         0.6733, 0.6463, 0.1087, 0.3436, 0.1590, 0.5244, 0.6336, 0.2801, 0.5600,
         0.8862, 0.0218, 0.4567, 0.5679, 0.1391, 0.8333, 0.0242, 0.1868, 0.7207,
         0.4460, 0.3564, 0.2982, 0.1674, 0.2275, 0.4009, 0.9885, 0.0856, 0.6432,
         0.6325, 0.4008, 0.9835, 0.9428, 0.8887, 0.2767, 0.1989, 0.2912, 0.3890,
         0.2165, 0.4106, 0.3160, 0.7425, 0.0022, 0.2575, 0.7383, 0.3709, 0.8586,
         0.3847, 0.7544, 0.4262, 0.7054, 0.7380, 0.1239, 0.6175, 0.7950, 0.7947,
         0.1139, 0.4005, 0.4960, 0.8954, 0.3701, 0.0450, 0.7517, 0.7035, 0.4438,
         0.8548, 0.4545, 0.7835, 0.7583, 0.3870, 0.2179, 0.7194, 0.5105, 0.5271,
         0.7799, 0.5073, 0.6938, 0.0642, 0.6874, 0.9541, 0.2668, 0.8696, 0.6239,
         0.3144, 0.7203, 0.1575, 0.2650, 0.7741, 0.7478, 0.2982, 0.3811, 0.6717,
         0.5547, 0.7600, 0.6651, 0.1935, 0.5417, 0.9170, 0.4534, 0.6567, 0.9738,
         0.4686, 0.0425, 0.9